### Goal of this program: To take inputs (stats from tennis players) and output predicted winning percentage

### Steps:
###### 1) Scrape data from tennis database (tennisabstract.com for now)
###### 2) input data into model
###### 3) Model ouputs predicted winning percentage (and maybe predicted outcome against opponent in the future)

In [1]:
import numpy as np
import random
from random import randint
from sklearn.preprocessing import MinMaxScaler
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPool2D, GlobalAveragePooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import *
from keras.models import Model
from keras.applications import imagenet_utils
from keras.preprocessing import image
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix
import itertools
# from keras.utils.vis_utils import plot_model
%matplotlib inline

import os
import json
# import nbimporter


In [2]:
# from serve_scraper import data

# import DataTransfer

In [3]:
with open("new_data.json", "r") as file:
    data = json.load(file)
print(data)

{'extra1': [130.58310396831504, 68.35364839419881, 17.03211720193373, 13.977882798066268, {'singles': [14.547882798066269, 299.0, 168.13, 72.32941399033135, 14], 'doubles': [15.17211720193373, 10.0, -11.24, 54.41058600966865, 2]}], 'extra2': [124.98326040695251, 72.26051599514942, 16.570171998383135, 15.280171998383137, {'singles': [15.850171998383138, 299.0, 170.13, 74.63914000808431, 13], 'doubles': [14.710171998383137, 8.0, -13.24, 52.10085999191569, 3]}], 'extra3': [124.07075931641963, 72.89714466296306, 15.51761844567898, 15.492381554321017, {'singles': [16.062381554321018, 297.0, 168.13, 73.57809222839491, 14], 'doubles': [14.922381554321017, 8.0, -11.24, 46.83809222839491, 2]}], 'extra4': [123.29382471331088, 73.43919206048078, 15.336935979839739, 15.67306402016026, {'singles': [14.61693597983974, 297.0, 168.13, 72.6746798991987, 10], 'doubles': [13.47693597983974, 10.0, -13.24, 45.9346798991987, 2]}], 'extra5': [127.50977265602786, 71.50216696932176, 16.317388989773917, 14.6926

### Setting all the training/valid sets up 

In [4]:
# Data dependent on match type (singles and doubles) will be put in corresponding lists. These corresponding lists
# will all be put together in another list. This list is then paired with another list in a dictionary. For example,
# singles training sample data will be put in their corresponding lists. These lists will be put in a singles training
# sample list, which countains all the lists with the singles training sample data. This singles training list will 
# then be paired with a doubles training sample list in a dictionary. 

In [5]:
adv_training_sets = dict()
scaled_adv_training_sets = dict()


valid_sets = dict()
scaled_valid_sets = dict()

singlesUtr_train_samples = []
doublesUtr_train_samples = []
servePercentage_train_samples = []

scaled_singlesUtr_train_samples = []
scaled_doublesUtr_train_samples = []
scaled_servePercentage_train_samples = []

basic_training_sets = []
basic_training_sets.append(singlesUtr_train_samples)
basic_training_sets.append(doublesUtr_train_samples) 
basic_training_sets.append(servePercentage_train_samples)

scaled_basic_training_sets = []
scaled_basic_training_sets.append(scaled_singlesUtr_train_samples) 
scaled_basic_training_sets.append(scaled_doublesUtr_train_samples) 
scaled_basic_training_sets.append(scaled_servePercentage_train_samples)

singlesGameDiff_train_samples = []
singlesWeightedGameDiff_train_samples = []
singlesWinPercentage_train_samples = []
singlesOpponentUtr_train_samples = []
singlesLongestWinStreak_train_samples = []

scaled_singlesGameDiff_train_samples = []
scaled_singlesWeightedGameDiff_train_samples = []
scaled_singlesWinPercentage_train_samples = []
scaled_singlesOpponentUtr_train_samples = []
scaled_singlesLongestWinStreak_train_samples = []

adv_training_sets["singles"] = [] 
adv_training_sets["singles"].append(singlesGameDiff_train_samples) 
adv_training_sets["singles"].append(singlesWeightedGameDiff_train_samples) 
adv_training_sets["singles"].append(singlesWinPercentage_train_samples) 
adv_training_sets["singles"].append(singlesOpponentUtr_train_samples) 
adv_training_sets["singles"].append(singlesLongestWinStreak_train_samples)

scaled_adv_training_sets["singles"] = [] 
scaled_adv_training_sets["singles"].append(scaled_singlesGameDiff_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesWeightedGameDiff_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesWinPercentage_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesOpponentUtr_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesLongestWinStreak_train_samples)


# print(scaled_adv_training_sets)

'''
for i in range(5):   # Adding scaled values to all advanced stat training sets
    # list_ = adv_training_sets[match_type][i]
    scaled_list = scaled_adv_training_sets['singles'][i]
    scaled_adv_training_sets['singles'][i].extend([0, i])
    # print(f"adv_training_set scaled list: {type(scaled_adv_training_sets[dict_string][i])}")

print(scaled_adv_training_sets)

print(type(scaled_singlesGameDiff_train_samples))
'''

# print(singles_train_lists)

doublesGameDiff_train_samples = []
doublesWeightedGameDiff_train_samples = []
doublesWinPercentage_train_samples = []
doublesOpponentUtr_train_samples = []
doublesLongestWinStreak_train_samples = []

scaled_doublesGameDiff_train_samples = []
scaled_doublesWeightedGameDiff_train_samples = []
scaled_doublesWinPercentage_train_samples = []
scaled_doublesOpponentUtr_train_samples = []
scaled_doublesLongestWinStreak_train_samples = []

'''
adv_training_sets["doubles"] = [doublesGameDiff_train_samples, doublesWeightedGameDiff_train_samples, doublesWinPercentage_train_samples, doublesOpponentUtr_train_samples, doublesLongestWinStreak_train_samples]
scaled_adv_training_sets["doubles"] = [scaled_doublesGameDiff_train_samples, scaled_doublesWeightedGameDiff_train_samples, scaled_doublesWinPercentage_train_samples, scaled_doublesOpponentUtr_train_samples, scaled_doublesLongestWinStreak_train_samples]
'''

adv_training_sets["doubles"] = [] 
adv_training_sets["doubles"].append(doublesGameDiff_train_samples) 
adv_training_sets["doubles"].append(doublesWeightedGameDiff_train_samples) 
adv_training_sets["doubles"].append(doublesWinPercentage_train_samples) 
adv_training_sets["doubles"].append(doublesOpponentUtr_train_samples) 
adv_training_sets["doubles"].append(doublesLongestWinStreak_train_samples)
                                    
scaled_adv_training_sets["doubles"] = [] 
scaled_adv_training_sets["doubles"].append(scaled_doublesGameDiff_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesWeightedGameDiff_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesWinPercentage_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesOpponentUtr_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesLongestWinStreak_train_samples)

serveSpeed_train_labels = []
scaled_serveSpeed_train_labels = []

##########################################################################################################################################
'''
singlesUtr_valid_samples = []
doublesUtr_valid_samples = []
servePercentage_valid_samples = []

singlesGameDiff_valid_samples = []
singlesWeightedGameDiff_valid_samples = []
singlesWinPercentage_valid_samples = []
singlesOpponentUtr_valid_samples = []
singlesLongestWinStreak_valid_samples = []

valid_sets["singles"] = [singlesGameDiff_valid_samples, singlesWeightedGameDiff_valid_samples, singlesWinPercentage_valid_samples, singlesOpponentUtr_valid_samples, singlesLongestWinStreak_valid_samples]
scaled_valid_sets["singles"] = [scaled_singlesGameDiff_valid_samples, scaled_singlesWeightedGameDiff_valid_samples, scaled_singlesWinPercentage_valid_samples, scaled_singlesOpponentUtr_valid_samples, scaled_singlesLongestWinStreak_valid_samples]


doublesGameDiff_valid_samples = []
doublesWeightedGameDiff_valid_samples = []
doublesWinPercentage_valid_samples = []
doublesOpponentUtr_valid_samples = []
doublesLongestWinStreak_valid_samples = []

valid_sets["doubles"] = [doublesGameDiff_valid_samples, doublesWeightedGameDiff_valid_samples, doublesWinPercentage_valid_samples, doublesOpponentUtr_valid_samples, singlesLongestWinStreak_valid_samples]
scaled_valid_sets["doubles"] = [scaled_doublesGameDiff_valid_samples, scaled_doublesWeightedGameDiff_valid_samples, scaled_doublesWinPercentage_valid_samples, scaled_doublesOpponentUtr_valid_samples, scaled_singlesLongestWinStreak_valid_samples]
'''


# serveSpeed_valid_labels = []

#######################################################################################################################################



'\nsinglesUtr_valid_samples = []\ndoublesUtr_valid_samples = []\nservePercentage_valid_samples = []\n\nsinglesGameDiff_valid_samples = []\nsinglesWeightedGameDiff_valid_samples = []\nsinglesWinPercentage_valid_samples = []\nsinglesOpponentUtr_valid_samples = []\nsinglesLongestWinStreak_valid_samples = []\n\nvalid_sets["singles"] = [singlesGameDiff_valid_samples, singlesWeightedGameDiff_valid_samples, singlesWinPercentage_valid_samples, singlesOpponentUtr_valid_samples, singlesLongestWinStreak_valid_samples]\nscaled_valid_sets["singles"] = [scaled_singlesGameDiff_valid_samples, scaled_singlesWeightedGameDiff_valid_samples, scaled_singlesWinPercentage_valid_samples, scaled_singlesOpponentUtr_valid_samples, scaled_singlesLongestWinStreak_valid_samples]\n\n\ndoublesGameDiff_valid_samples = []\ndoublesWeightedGameDiff_valid_samples = []\ndoublesWinPercentage_valid_samples = []\ndoublesOpponentUtr_valid_samples = []\ndoublesLongestWinStreak_valid_samples = []\n\nvalid_sets["doubles"] = [doub

In [6]:
adv_testing_sets = dict()
scaled_adv_testing_sets = dict()


singlesUtr_test_samples = []
doublesUtr_test_samples = []
servePercentage_test_samples = []

scaled_singlesUtr_test_samples = []
scaled_doublesUtr_test_samples = []
scaled_servePercentage_test_samples = []

basic_testing_sets = []
basic_testing_sets.append(singlesUtr_test_samples)
basic_testing_sets.append(doublesUtr_test_samples) 
basic_testing_sets.append(servePercentage_test_samples)

scaled_basic_testing_sets = []
scaled_basic_testing_sets.append(scaled_singlesUtr_test_samples) 
scaled_basic_testing_sets.append(scaled_doublesUtr_test_samples) 
scaled_basic_testing_sets.append(scaled_servePercentage_test_samples)

singlesGameDiff_test_samples = []
singlesWeightedGameDiff_test_samples = []
singlesWinPercentage_test_samples = []
singlesOpponentUtr_test_samples = []
singlesLongestWinStreak_test_samples = []

scaled_singlesGameDiff_test_samples = []
scaled_singlesWeightedGameDiff_test_samples = []
scaled_singlesWinPercentage_test_samples = []
scaled_singlesOpponentUtr_test_samples = []
scaled_singlesLongestWinStreak_test_samples = []

adv_testing_sets["singles"] = [] 
adv_testing_sets["singles"].append(singlesGameDiff_test_samples) 
adv_testing_sets["singles"].append(singlesWeightedGameDiff_test_samples) 
adv_testing_sets["singles"].append(singlesWinPercentage_test_samples) 
adv_testing_sets["singles"].append(singlesOpponentUtr_test_samples) 
adv_testing_sets["singles"].append(singlesLongestWinStreak_test_samples)

scaled_adv_testing_sets["singles"] = [] 
scaled_adv_testing_sets["singles"].append(scaled_singlesGameDiff_test_samples) 
scaled_adv_testing_sets["singles"].append(scaled_singlesWeightedGameDiff_test_samples) 
scaled_adv_testing_sets["singles"].append(scaled_singlesWinPercentage_test_samples) 
scaled_adv_testing_sets["singles"].append(scaled_singlesOpponentUtr_test_samples) 
scaled_adv_testing_sets["singles"].append(scaled_singlesLongestWinStreak_test_samples)


# print(scaled_adv_testing_sets)

'''
for i in range(5):   # Adding scaled values to all advanced stat testing sets
    # list_ = adv_testing_sets[match_type][i]
    scaled_list = scaled_adv_testing_sets['singles'][i]
    scaled_adv_testing_sets['singles'][i].extend([0, i])
    # print(f"adv_testing_set scaled list: {type(scaled_adv_testing_sets[dict_string][i])}")

print(scaled_adv_testing_sets)

print(type(scaled_singlesGameDiff_test_samples))
'''

# print(singles_test_lists)

doublesGameDiff_test_samples = []
doublesWeightedGameDiff_test_samples = []
doublesWinPercentage_test_samples = []
doublesOpponentUtr_test_samples = []
doublesLongestWinStreak_test_samples = []

scaled_doublesGameDiff_test_samples = []
scaled_doublesWeightedGameDiff_test_samples = []
scaled_doublesWinPercentage_test_samples = []
scaled_doublesOpponentUtr_test_samples = []
scaled_doublesLongestWinStreak_test_samples = []

'''
adv_testing_sets["doubles"] = [doublesGameDiff_test_samples, doublesWeightedGameDiff_test_samples, doublesWinPercentage_test_samples, doublesOpponentUtr_test_samples, doublesLongestWinStreak_test_samples]
scaled_adv_testing_sets["doubles"] = [scaled_doublesGameDiff_test_samples, scaled_doublesWeightedGameDiff_test_samples, scaled_doublesWinPercentage_test_samples, scaled_doublesOpponentUtr_test_samples, scaled_doublesLongestWinStreak_test_samples]
'''

adv_testing_sets["doubles"] = [] 
adv_testing_sets["doubles"].append(doublesGameDiff_test_samples) 
adv_testing_sets["doubles"].append(doublesWeightedGameDiff_test_samples) 
adv_testing_sets["doubles"].append(doublesWinPercentage_test_samples) 
adv_testing_sets["doubles"].append(doublesOpponentUtr_test_samples) 
adv_testing_sets["doubles"].append(doublesLongestWinStreak_test_samples)
                                    
scaled_adv_testing_sets["doubles"] = [] 
scaled_adv_testing_sets["doubles"].append(scaled_doublesGameDiff_test_samples) 
scaled_adv_testing_sets["doubles"].append(scaled_doublesWeightedGameDiff_test_samples) 
scaled_adv_testing_sets["doubles"].append(scaled_doublesWinPercentage_test_samples) 
scaled_adv_testing_sets["doubles"].append(scaled_doublesOpponentUtr_test_samples) 
scaled_adv_testing_sets["doubles"].append(scaled_doublesLongestWinStreak_test_samples)

serveSpeed_test_labels = []
scaled_serveSpeed_test_labels = []

In [7]:
print(len(data))

2550


### Getting Training and Varification sets data from data dictionary

In [8]:
# Need to do: get all data from data dict into training_labels and training_sameples
# Each stat can go in its individual list, I will just have to be careful that every individual number (Stat) for each player is the same 
# index in each list. For example, Sinner's serve percentage will have the same index as his singles UTR, doubles UTR, etc. 

In [9]:
# PS - only lists with a string ("no data?") are the serve speed lists!

name_list = list(data)

match_types = ["singles", "doubles"]

data_len = len(data)

training_data_len = data_len - 10
test_data_len = 10

# Training Set
for i in range(training_data_len):
    
    training_list = []
    name = random.choice(name_list)
    name_list.remove(name)
    player = data[name]
    # print(player)

    serveSpeed_train_labels.append(player[0])   # appending serve speed to label list

    for i in range(3):
        # print(player[i+1])
        if player[i+1] != 0:
            basic_training_sets[i].append(player[i+1])  # appending serve percentage, singles utr, and doubles utr to training set lists
        else:
            basic_training_sets[i].append(None)    # If singles or doubles UTR, or serve percentage equal 0, append None to the training set
    '''   
    servePercentage_train_samples.append(player[1])
    singlesUtr_train_samples.append(player[2])
    doublesUtr_train_samples.append(player[3])
    '''

    for match_type in match_types:  
        dict_string = f"{match_type}"
        match_type_data = player[4][dict_string]
        # print(match_type_data)
        if match_type_data == None:    # for singes and doubles stats, if no activity is recorded, put None for stats variables
            for stats_list in adv_training_sets[dict_string]:
                stats_list.append(None)
        else:
            for i in range(5): # Append data from data dict to appropriate training set lists
                adv_training_sets[dict_string][i].append(match_type_data[i])  


###### Testing Set #######################################################################################################################

print(f"Remaining name_list: {name_list}")

for name in name_list:
    print(name)
    # testing_list = []
    # name = random.choice(name_list)
    # name_list.remove(name)
    player = data[name]
    # print(player)

    serveSpeed_test_labels.append(player[0])   # appending serve speed to label list

    for i in range(3):
        # print(player[i+1])
        if player[i+1] != 0:
            basic_testing_sets[i].append(player[i+1])  # appending serve percentage, singles utr, and doubles utr to training set lists
        else:
            basic_testing_sets[i].append(None)    # If singles or doubles UTR, or serve percentage equal 0, append None to the training set
    '''   
    servePercentage_train_samples.append(player[1])
    singlesUtr_train_samples.append(player[2])
    doublesUtr_train_samples.append(player[3])
    '''

    for match_type in match_types:  
        dict_string = f"{match_type}"
        match_type_data = player[4][dict_string]
        print(match_type_data)
        if match_type_data == None:    # for singes and doubles stats, if no activity is recorded, put None for stats variables
            for stats_list in adv_testing_sets[dict_string]:
                stats_list.append(None)
        else:
            for i in range(5): # Append data from data dict to appropriate training set lists
                adv_testing_sets[dict_string][i].append(match_type_data[i]) 
        
print(f"Final name list: {name_list}")
# Validation Set ##################################################################################################################
'''
for i in range(10):
    valid_list = []
    name = random.choice(name_list)
    name_list.remove(name)
    player = data[name]
    # print(player)

    serveSpeed_valid_labels.append(player[0])

    servePercentage_valid_samples.append(player[1])
    singlesUtr_valid_samples.append(player[2])
    doublesUtr_valid_samples.append(player[3])

    for match_type in match_types:  
        dict_string = f"{match_type}"
        match_type_data = player[4][dict_string]
        
        if match_type_data == None:    # for singes and doubles stats, if no activity is recorded, put None for stats variables
            for stats_list in valid_sets[dict_string]:
                stats_list.append(None)
        else:
            for i in range(5): 
                valid_sets[dict_string][i].append(match_type_data[i])              
            
'''
    


print(f"Players left for test set: {len(name_list)}")


Remaining name_list: ['extra50', 'extra203', 'extra884', 'extra1441', 'extra1779', 'extra1945', 'extra2036', 'extra2195', 'extra2376', 'extra2408']
extra50
[14.846414126895846, 299.0, 170.13, 79.65792936552077, 10]
[14.873585873104153, 10.0, -11.24, 52.91792936552077, 2]
extra203
[15.80358028600358, 77.0, 3.01, 64.38790143001789, 6]
[13.68641971399642, -22.0, -52.49, 41.0620985699821, 4]
extra884
[16.101348019008878, 145.0, 98.42, 70.91674009504439, 11]
[14.801348019008877, 6.0, -11.7, 43.25325990495561, 5]
extra1441
[14.84822248270515, -24.0, 37.62, 43.94888758647425, 4]
[11.61177751729485, 3.0, -15.1, 80.49111241352576, 3]
extra1779
[15.8820450554342, 69.0, 14.05, 55.93977472282901, 3]
[13.667954944565802, 0.0, -14.76, 43.21022527717099, 4]
extra1945
[2.6295726770676056, 26.0, 19.1, 62.702136614661974, 2]
[2.4595726770676056, -4.0, -5.7, 52.702136614661974, 3]
extra2036
[6.262314592694261, 1.0, -7.6, 48.688427036528694, 3]
[7.012314592694261, 6.0, 2.2, 48.688427036528694, 1]
extra219

In [10]:
print(name_list)

['extra50', 'extra203', 'extra884', 'extra1441', 'extra1779', 'extra1945', 'extra2036', 'extra2195', 'extra2376', 'extra2408']


In [11]:
print(f"elements in data: {len(data)}")
print(f"Elements left to use for test set: {len(name_list)}")

elements in data: 2550
Elements left to use for test set: 10


In [12]:
### Scalers

In [13]:
scaler = MinMaxScaler(feature_range=(0,1))
sample_scaler = MinMaxScaler(feature_range=(0,1)) # work these into training and testing sets maybe ??? if not then delete
label_scaler = MinMaxScaler(feature_range=(0,1))

print(scaled_basic_training_sets)
def scaler_func(list_, scaled_list, scaler):

    # list_ = np.array(list_)
    '''
    for elem in list_:
        if elem != 'No data?' and elem != None:
            elem = float(elem)
        else:
            elem = np.array([0])
    '''
    
    # list_ = [float(x) if x != 'No data?' and x != None else np.array([0]) for x in list_]

    # print(list_)
    numeric_data = [x if isinstance(x, (int, float)) else 0 for x in list_]  # creates a new list numeric_data that contains all values that are intagers and floats
    numeric_data = np.array(numeric_data).reshape(-1, 1)  # Converts all numbers in training set to numpy. Training sets with None/string values can't be converted to numpy (I think)
    scaled_numeric_data = scaler.fit_transform((numeric_data))
    
    # scaled_list = scaled_basic_training_sets[i]
    numeric_index = 0

    '''
    for item in list_:     # If item in basic_training_sets list is string or None, append that. If item is int or float, append next scaled_numeric_data value 
        if isinstance(item, (int, float)):
            scaled_list.append(scaled_numeric_data[numeric_index])
            numeric_index += 1
        else:
            scaled_list.append(np.array([0]))
    '''

    # print(f"scaled_list = {scaled_list}")

    # scaled_list.clear()

    # print(scaled_list)
    '''
    if scaled_list != None:
        scaled_list[:] = scaled_numeric_data.reshape(70,1) # manually changed the shape
    else:
        scaled_list = scaled_numeric_data.flatten()
    '''

    # scaled_list = [0 if x != 'No data?' or x != None else x for x in scaled_list]
    
    # print(f"scaled_numeric_data = {scaled_numeric_data}")

    return scaled_numeric_data
            


'''
for i in range(3):   # Adding scaled values to all basic training sets
    list_ = basic_training_sets[i]
    scaled_list = scaled_basic_training_sets[i]

    print(f"pre-scaled scaled_basic_training_sets[0] = {type(scaled_basic_training_sets[0])}")

    scaled_basic_training_sets[i] = scaler_func(basic_training_sets[i], scaled_basic_training_sets[i])

    # scaled_basic_training_sets[i] = np.array(scaled_basic_training_sets[i]).reshape(-1, 1)

    print(f"basic_training_set scaled list type: {type(scaled_basic_training_sets[i])}")
    print(f"basic_training_set scaled list: {scaled_basic_training_sets[i]}")


print(f"scaled_singlesUtr_train_samples type = {type(scaled_singlesUtr_train_samples)}")
print(f"scaled_singlesUtr_train_samples = {scaled_singlesUtr_train_samples}")


for match_type in match_types:  
    dict_string = f"{match_type}"
    dict_string = 'singles'    # CHANGE LATER 
    # match_type_data = player[4][dict_string]

    for i in range(5):   # Adding scaled values to all advanced stat training sets
        list_ = adv_training_sets[match_type][i]
        scaled_list = scaled_adv_training_sets[dict_string][i]
        scaled_adv_training_sets[dict_string][i].extend(scaler_func(list_, scaled_list))
        # scaled_adv_training_sets[dict_string][i] = scaler.fit_transform((scaled_adv_training_sets[dict_string][i]).reshape(-1,1))
        print(f"adv_training_set scaled list: {type(scaled_adv_training_sets[dict_string][i])}")

      
'''


[[], [], []]


'\nfor i in range(3):   # Adding scaled values to all basic training sets\n    list_ = basic_training_sets[i]\n    scaled_list = scaled_basic_training_sets[i]\n\n    print(f"pre-scaled scaled_basic_training_sets[0] = {type(scaled_basic_training_sets[0])}")\n\n    scaled_basic_training_sets[i] = scaler_func(basic_training_sets[i], scaled_basic_training_sets[i])\n\n    # scaled_basic_training_sets[i] = np.array(scaled_basic_training_sets[i]).reshape(-1, 1)\n\n    print(f"basic_training_set scaled list type: {type(scaled_basic_training_sets[i])}")\n    print(f"basic_training_set scaled list: {scaled_basic_training_sets[i]}")\n\n\nprint(f"scaled_singlesUtr_train_samples type = {type(scaled_singlesUtr_train_samples)}")\nprint(f"scaled_singlesUtr_train_samples = {scaled_singlesUtr_train_samples}")\n\n\nfor match_type in match_types:  \n    dict_string = f"{match_type}"\n    dict_string = \'singles\'    # CHANGE LATER \n    # match_type_data = player[4][dict_string]\n\n    for i in range(5): 

### Assigning values to training and testing samples

In [14]:
# serveSpeed_train_labels = np.array(serveSpeed_train_labels)

print(f"pre-scaled serve speed training labels: {type(scaled_serveSpeed_train_labels)}")

scaled_serveSpeed_train_labels = scaler_func(serveSpeed_train_labels, scaled_serveSpeed_train_labels, label_scaler)
# scaled_serveSpeed_train_labels = np.array(serveSpeed_train_labels).reshape(-1, 1)

scaled_singlesUtr_train_samples = scaler_func(singlesUtr_train_samples, scaled_singlesUtr_train_samples, sample_scaler)

scaled_doublesUtr_train_samples = scaler_func(doublesUtr_train_samples, scaled_doublesUtr_train_samples, sample_scaler)

scaled_servePercentage_train_samples = scaler_func(servePercentage_train_samples, scaled_servePercentage_train_samples, sample_scaler)

scaled_singlesGameDiff_train_samples = scaler_func(singlesGameDiff_train_samples, scaled_singlesGameDiff_train_samples, sample_scaler)

scaled_singlesWeightedGameDiff_train_samples = scaler_func(singlesWeightedGameDiff_train_samples, scaled_singlesWeightedGameDiff_train_samples, sample_scaler)

scaled_singlesWinPercentage_train_samples = scaler_func(singlesWinPercentage_train_samples, scaled_singlesWinPercentage_train_samples, sample_scaler)

scaled_singlesOpponentUtr_train_samples = scaler_func(singlesOpponentUtr_train_samples, scaled_singlesOpponentUtr_train_samples, sample_scaler)

scaled_singlesLongestWinStreak_train_samples = scaler_func(singlesLongestWinStreak_train_samples, scaled_singlesLongestWinStreak_train_samples, sample_scaler)

print(f"Scaled serve speed training lables: {scaled_serveSpeed_train_labels}")

pre-scaled serve speed training labels: <class 'list'>
Scaled serve speed training lables: [[0.86295608]
 [0.80541975]
 [0.78826272]
 ...
 [0.28294237]
 [0.28344955]
 [0.77876142]]


In [15]:
print(f"pre-scaled serve speed testing labels: {type(scaled_serveSpeed_test_labels)}")

scaled_serveSpeed_test_labels = scaler_func(serveSpeed_test_labels, scaled_serveSpeed_test_labels, label_scaler)
# scaled_serveSpeed_test_labels = np.array(scaled_serveSpeed_test_labels).reshape(-1, 1)

scaled_singlesUtr_test_samples = scaler_func(singlesUtr_test_samples, scaled_singlesUtr_test_samples, sample_scaler)

scaled_doublesUtr_test_samples = scaler_func(doublesUtr_test_samples, scaled_doublesUtr_test_samples, sample_scaler)

scaled_servePercentage_test_samples = scaler_func(servePercentage_test_samples, scaled_servePercentage_test_samples, sample_scaler)

scaled_singlesGameDiff_test_samples = scaler_func(singlesGameDiff_test_samples, scaled_singlesGameDiff_test_samples, sample_scaler)

scaled_singlesWeightedGameDiff_test_samples = scaler_func(singlesWeightedGameDiff_test_samples, scaled_singlesWeightedGameDiff_test_samples, sample_scaler)

scaled_singlesWinPercentage_test_samples = scaler_func(singlesWinPercentage_test_samples, scaled_singlesWinPercentage_test_samples, sample_scaler)

scaled_singlesOpponentUtr_test_samples = scaler_func(singlesOpponentUtr_test_samples, scaled_singlesOpponentUtr_test_samples, sample_scaler)

scaled_singlesLongestWinStreak_test_samples = scaler_func(singlesLongestWinStreak_test_samples, scaled_singlesLongestWinStreak_test_samples, sample_scaler)

print(f"Scaled serve speed testing lables: {scaled_serveSpeed_test_labels}")

pre-scaled serve speed testing labels: <class 'list'>
Scaled serve speed testing lables: [[0.91486123]
 [1.        ]
 [0.73922663]
 [0.78320668]
 [0.77714531]
 [0.06000567]
 [0.1968842 ]
 [0.10112847]
 [0.12408438]
 [0.        ]]


In [16]:
'''

print(scaled_serveSpeed_train_labels.shape)
print(scaled_singlesUtr_train_samples.shape)
print(scaled_doublesUtr_train_samples.shape)
print(scaled_servePercentage_train_samples.shape)
print(scaled_singlesGameDiff_train_samples.shape)
print(scaled_singlesWeightedGameDiff_train_samples.shape)
print(scaled_singlesWinPercentage_train_samples.shape)
print(scaled_singlesOpponentUtr_train_samples.shape)
print(scaled_singlesLongestWinStreak_train_samples.shape)



print(len(scaled_singlesUtr_train_samples))
print(len(scaled_doublesUtr_train_samples))
print(len(scaled_servePercentage_train_samples))
print(len(scaled_singlesGameDiff_train_samples))
print(len(scaled_singlesWeightedGameDiff_train_samples))
print(len(scaled_singlesWinPercentage_train_samples))
print(len(scaled_singlesOpponentUtr_train_samples))
print(len(scaled_singlesLongestWinStreak_train_samples))

'''

print(type(scaled_singlesUtr_train_samples))
print(type(scaled_doublesUtr_train_samples))
print(type(scaled_servePercentage_train_samples))
print(type(scaled_singlesGameDiff_train_samples))
print(type(scaled_singlesWeightedGameDiff_train_samples))
print(type(scaled_singlesWinPercentage_train_samples))
print(type(scaled_singlesOpponentUtr_train_samples))
print(type(scaled_singlesLongestWinStreak_train_samples))
print(type(scaled_serveSpeed_train_labels))


'''

print(scaled_singlesUtr_train_samples)
print(scaled_doublesUtr_train_samples)
print(scaled_servePercentage_train_samples)
print(scaled_singlesGameDiff_train_samples)
print(scaled_singlesWeightedGameDiff_train_samples)
print(scaled_singlesWinPercentage_train_samples)
print(scaled_singlesOpponentUtr_train_samples)
print(scaled_singlesLongestWinStreak_train_samples)
print(scaled_serveSpeed_train_labels)
'''

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


'\n\nprint(scaled_singlesUtr_train_samples)\nprint(scaled_doublesUtr_train_samples)\nprint(scaled_servePercentage_train_samples)\nprint(scaled_singlesGameDiff_train_samples)\nprint(scaled_singlesWeightedGameDiff_train_samples)\nprint(scaled_singlesWinPercentage_train_samples)\nprint(scaled_singlesOpponentUtr_train_samples)\nprint(scaled_singlesLongestWinStreak_train_samples)\nprint(scaled_serveSpeed_train_labels)\n'

In [17]:
'''

print(scaled_serveSpeed_test_labels.shape)
print(scaled_singlesUtr_test_samples.shape)
print(scaled_doublesUtr_test_samples.shape)
print(scaled_servePercentage_test_samples.shape)
print(scaled_singlesGameDiff_test_samples.shape)
print(scaled_singlesWeightedGameDiff_test_samples.shape)
print(scaled_singlesWinPercentage_test_samples.shape)
print(scaled_singlesOpponentUtr_test_samples.shape)
print(scaled_singlesLongestWinStreak_test_samples.shape)



print(len(scaled_singlesUtr_test_samples))
print(len(scaled_doublesUtr_test_samples))
print(len(scaled_servePercentage_test_samples))
print(len(scaled_singlesGameDiff_test_samples))
print(len(scaled_singlesWeightedGameDiff_test_samples))
print(len(scaled_singlesWinPercentage_test_samples))
print(len(scaled_singlesOpponentUtr_test_samples))
print(len(scaled_singlesLongestWinStreak_test_samples))

'''

print(type(scaled_singlesUtr_test_samples))
print(type(scaled_doublesUtr_test_samples))
print(type(scaled_servePercentage_test_samples))
print(type(scaled_singlesGameDiff_test_samples))
print(type(scaled_singlesWeightedGameDiff_test_samples))
print(type(scaled_singlesWinPercentage_test_samples))
print(type(scaled_singlesOpponentUtr_test_samples))
print(type(scaled_singlesLongestWinStreak_test_samples))
print(type(scaled_serveSpeed_test_labels))


'''

print(scaled_singlesUtr_test_samples)
print(scaled_doublesUtr_test_samples)
print(scaled_servePercentage_test_samples)
print(scaled_singlesGameDiff_test_samples)
print(scaled_singlesWeightedGameDiff_test_samples)
print(scaled_singlesWinPercentage_test_samples)
print(scaled_singlesOpponentUtr_test_samples)
print(scaled_singlesLongestWinStreak_test_samples)
print(scaled_serveSpeed_test_labels)
'''

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


'\n\nprint(scaled_singlesUtr_test_samples)\nprint(scaled_doublesUtr_test_samples)\nprint(scaled_servePercentage_test_samples)\nprint(scaled_singlesGameDiff_test_samples)\nprint(scaled_singlesWeightedGameDiff_test_samples)\nprint(scaled_singlesWinPercentage_test_samples)\nprint(scaled_singlesOpponentUtr_test_samples)\nprint(scaled_singlesLongestWinStreak_test_samples)\nprint(scaled_serveSpeed_test_labels)\n'

In [18]:
print(scaled_serveSpeed_test_labels)

[[0.91486123]
 [1.        ]
 [0.73922663]
 [0.78320668]
 [0.77714531]
 [0.06000567]
 [0.1968842 ]
 [0.10112847]
 [0.12408438]
 [0.        ]]


### Making the Model

In [19]:
end_sets = np.column_stack((scaled_singlesUtr_train_samples, scaled_doublesUtr_train_samples, scaled_servePercentage_train_samples,
                           scaled_singlesGameDiff_train_samples, scaled_singlesWeightedGameDiff_train_samples, 
                            scaled_singlesWinPercentage_train_samples, scaled_singlesOpponentUtr_train_samples, 
                            scaled_singlesLongestWinStreak_train_samples))

end_sets = end_sets[:, :, np.newaxis]  # Adds a third dimension

print(end_sets)
# print(scaled_singlesUtr_train_samples)




[[[0.73814685]
  [0.93839781]
  [0.91737961]
  ...
  [0.62951938]
  [0.73961872]
  [0.38709677]]

 [[0.58648886]
  [0.89772359]
  [0.86702141]
  ...
  [0.08216064]
  [0.42855508]
  [0.16129032]]

 [[0.60533006]
  [0.86408857]
  [0.88651114]
  ...
  [0.17441546]
  [0.39209493]
  [0.06451613]]

 ...

 [[0.39475329]
  [0.36400901]
  [0.47541457]
  ...
  [0.15260338]
  [0.0942051 ]
  [0.        ]]

 [[0.32346196]
  [0.4664398 ]
  [0.51695035]
  ...
  [0.14610847]
  [0.43035254]
  [0.06451613]]

 [[0.65517918]
  [0.88258259]
  [0.84044369]
  ...
  [0.14521541]
  [0.57434267]
  [0.12903226]]]


In [20]:
test_sets = np.column_stack((scaled_singlesUtr_test_samples, scaled_doublesUtr_test_samples, scaled_servePercentage_test_samples,
                           scaled_singlesGameDiff_test_samples, scaled_singlesWeightedGameDiff_test_samples, 
                            scaled_singlesWinPercentage_test_samples, scaled_singlesOpponentUtr_test_samples, 
                            scaled_singlesLongestWinStreak_test_samples))

test_sets = test_sets[:, :, np.newaxis] # Adds a third dimension

print(len(test_sets))
print(scaled_singlesUtr_test_samples)

10
[[1.        ]
 [0.81583452]
 [0.66593271]
 [0.53690455]
 [0.50497758]
 [0.74438942]
 [0.94924521]
 [0.81599925]
 [0.36634615]
 [0.        ]]


In [21]:

model = Sequential([
    Input(shape=(8,)),
    Dense(64, activation='relu'),
    Dropout(0.1),
    BatchNormalization(),
    
    Dense(32, activation='relu'),
    Dropout(0.1),
    BatchNormalization(),
    
    Dense(16, activation='relu'),
    
    Dense(1)
])


### Training the Model

In [22]:
model.compile(Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae'])

In [23]:
model.fit(end_sets, scaled_serveSpeed_train_labels, validation_split=0.1, batch_size=10, epochs=1000, shuffle=True, verbose=2)

Epoch 1/1000
229/229 - 1s - 3ms/step - loss: 0.5272 - mae: 0.5720 - val_loss: 0.1703 - val_mae: 0.3840
Epoch 2/1000
229/229 - 0s - 433us/step - loss: 0.3438 - mae: 0.4621 - val_loss: 0.1589 - val_mae: 0.3507
Epoch 3/1000
229/229 - 0s - 426us/step - loss: 0.2555 - mae: 0.4049 - val_loss: 0.0986 - val_mae: 0.2720
Epoch 4/1000
229/229 - 0s - 424us/step - loss: 0.1981 - mae: 0.3541 - val_loss: 0.0643 - val_mae: 0.2132
Epoch 5/1000
229/229 - 0s - 421us/step - loss: 0.1790 - mae: 0.3358 - val_loss: 0.0592 - val_mae: 0.2053
Epoch 6/1000
229/229 - 0s - 566us/step - loss: 0.1529 - mae: 0.3096 - val_loss: 0.0559 - val_mae: 0.1999
Epoch 7/1000
229/229 - 0s - 427us/step - loss: 0.1412 - mae: 0.2962 - val_loss: 0.0452 - val_mae: 0.1797
Epoch 8/1000
229/229 - 0s - 425us/step - loss: 0.1217 - mae: 0.2730 - val_loss: 0.0423 - val_mae: 0.1715
Epoch 9/1000
229/229 - 0s - 424us/step - loss: 0.1085 - mae: 0.2601 - val_loss: 0.0402 - val_mae: 0.1658
Epoch 10/1000
229/229 - 0s - 425us/step - loss: 0.0936 - 

In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,373 (40.52 KB)

 Trainable params: 3,393 (13.25 KB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 6,788 (26.52 KB)

### Preprocessing Test Data

In [25]:
predictions = model.predict(test_sets, batch_size=1, verbose=0)
print(predictions)

[[ 0.9029693 ]
 [ 1.0328716 ]
 [ 0.7981539 ]
 [ 0.8052837 ]
 [ 0.8076811 ]
 [-0.11498863]
 [ 0.38830775]
 [ 0.2315008 ]
 [ 0.29005927]
 [ 0.17355555]]


In [26]:
# model(model, to_file='model.png', show_shapes=True)

In [27]:
# print(f"confidence\n")
print(len(predictions))

print(scaled_serveSpeed_test_labels)


original_speeds = label_scaler.inverse_transform(predictions)
print("og speeds")
print(original_speeds)



for i in range(len(predictions)):
    # label = scaled_serveSpeed_test_labels[i]
    name = name_list[i]
    real_speed = data[name][0]
    prediction = predictions[i]
    # self_scale = prediction * 
    print(f"{name}: Guess = {prediction}, Speed = {real_speed}, Prediction speed = {original_speeds[i]}")
print()

10
[[0.91486123]
 [1.        ]
 [0.73922663]
 [0.78320668]
 [0.77714531]
 [0.06000567]
 [0.1968842 ]
 [0.10112847]
 [0.12408438]
 [0.        ]]
og speeds
[[128.36076 ]
 [138.61423 ]
 [120.087456]
 [120.65024 ]
 [120.839455]
 [ 48.01116 ]
 [ 87.73743 ]
 [ 75.36031 ]
 [ 79.98246 ]
 [ 70.78657 ]]
extra50: Guess = [0.9029693], Speed = 129.29941925434787, Prediction speed = [128.36076]
extra203: Guess = [1.0328716], Speed = 136.0196047701846, Prediction speed = [138.61423]
extra884: Guess = [0.7981539], Speed = 115.43620351826182, Prediction speed = [120.087456]
extra1441: Guess = [0.8052837], Speed = 118.90764332436785, Prediction speed = [120.65024]
extra1779: Guess = [0.8076811], Speed = 118.42920626163294, Prediction speed = [120.839455]
extra1945: Guess = [-0.11498863], Speed = 61.82383748860929, Prediction speed = [48.01116]
extra2036: Guess = [0.38830775], Speed = 72.62795274858532, Prediction speed = [87.73743]
extra2195: Guess = [0.2315008], Speed = 65.06974799964459, Prediction sp